In [1]:
%cd ..

/home/hunar/SemCLIP


/home/hunar/SemCLIP/svt-llava-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# !pip3 install -r requirements.txt
# !pip3 install supervision torch tqdm fire datasets opencv-python openai-clip huggingface-hub torch python-dotenv clip torchvision Pillow pandas numpy matplotlib transformers
# !pip install --upgrade notebook
# !pip install --upgrade ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [3]:
# !bash sam_model_setup.sh
# !pip3 install -q 'git+https://github.com/facebookresearch/segment-anything.git'

In [2]:
# from SemCLIP.semclip import SemCLIP
from SemCLIP.image_utils import DEVICE, create_batches, pil_to_cv2
from SemCLIP.model_utils import convert_models_to_fp32, convert_models_to_fp16


# semclip = SemCLIP(model_name="openai/clip-vit-base-patch32", pool_type='attention', projection_dim=512, device=DEVICE)

In [3]:
import torch

from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

from config import dataset_mapper

dataset = load_dataset(dataset_mapper['COCO-13k'])

In [4]:
import torch 

from SemCLIP.semclip import SemCLIP
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

from PIL import Image


# Load the model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

device = "cuda:1" if torch.cuda.is_available() else "cpu"

clip_model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

## Finetune Model without downloading images locally [with HF data]

In [34]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
import wandb
from tqdm import tqdm

BATCH_SIZE = 64

wandb.login(key=os.getenv("WANDB_API_KEY"))

device = "cuda:1" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

class ImageCaptionDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        image = self.processor(images=example['downloaded_img'], return_tensors="pt")['pixel_values'].squeeze(0)
        caption = self.processor.tokenizer(example['caption'], return_tensors="pt", truncation=True, padding="max_length", max_length=77)['input_ids'].squeeze(0)
        return image, caption

# Load the dataset
dataset = load_dataset("hunarbatra/CLIP-LLaVA-Instruct-COCO-13k")
train_dataset = ImageCaptionDataset(dataset['train'])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

learning_rate = 1e-6 # or 1e-7
betas = (0.9, 0.98)
epsilon = 1e-6
weight_decay = 0.001 # originally 0.2
num_epochs = 1
batch_size = BATCH_SIZE

wandb_config = {
    "learning_rate": learning_rate,
    "betas": betas,
    "epsilon": epsilon,
    "weight_decay": weight_decay,
    "num_epochs": num_epochs,
    "batch_size": batch_size,
}

train_name = 'clip-v1-test-64'

optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=betas, eps=epsilon, weight_decay=weight_decay)

wandb.init(project="semclip", name=train_name, config=wandb_config)

# Training loop
for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        images, captions = batch
        images = images.to(device)
        captions = captions.to(device)
        
        outputs = model(pixel_values=images, input_ids=captions)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text
        
        ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
        
        total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
        total_loss.backward()
        
        if device == "cpu":
            optimizer.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            model.to(device)
            convert_models_to_fp16(clip_model)
        
        # Log the loss to wandb
        wandb.log({"Loss": total_loss.item()})
        
        # Update the progress bar description with the current loss
        progress_bar.set_postfix({"Loss": total_loss.item()})


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Loss,▃▅▃▇▅▂▆▃█▃█▂▄▂▂▄▅▄▄▄▄▂▃▂▄▃▂▂▂▂▃▃▄▃▃▃▁▅▃▂
Loss,0.28157


In [35]:
from dotenv import load_dotenv
from huggingface_hub import HfApi, Repository


load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

def upload_model_to_hf_hub(model, processor, model_name, hf_name):
    api = HfApi()

    repo_exists = api.repo_exists(repo_id=f"{hf_name}/{model_name}", token=HF_TOKEN)
    print(f"Repository exists: {repo_exists}")

    if repo_exists:
        repo = Repository(local_dir=model_name, clone_from=f"https://huggingface.co/{hf_name}/{model_name}", token=HF_TOKEN)
        commit_message = "Update model files"
    else:
        repo_url = api.create_repo(repo_id=model_name, token=HF_TOKEN, private=True)
        repo = Repository(local_dir=model_name, clone_from=repo_url, use_auth_token=HF_TOKEN)
        commit_message = "Add model files"

    model.save_pretrained(model_name)
    model.config.save_pretrained(model_name)
    # tokenizer.save_pretrained(model_name)
    processor.save_pretrained(model_name)
    
    repo.push_to_hub(commit_message=commit_message)
        
upload_model_to_hf_hub(model, processor, model_name='clip-v1-test', hf_name='hunarbatra')

For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
/home/hunar/SemCLIP/clip-v1-test is already a clone of https://huggingface.co/hunarbatra/clip-v1-test. Make sure you pull the latest changes with `repo.git_pull()`.


Repository exists: True


Upload file model.safetensors:   0%|          | 1.00/577M [00:00<?, ?B/s]

To https://huggingface.co/hunarbatra/clip-v1-test
   143eeef..47d3c3b  main -> main



In [36]:
!python3 evaluate.py --model_name='clip-v1-test' --data='COCO-13k' --batch_size=32 --data_subset=100 --eval_run_name='test6' --max_batch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


model.safetensors: 100%|█████████████████████| 605M/605M [01:21<00:00, 7.42MB/s]
  0%|                                                     | 0/1 [00:00<?, ?it/s]Batch 0: Zero-shot top-1 Accuracy: 68.75%, std err: 8.19%
Batch 0: Avg Cosine Similarity Score: 29.167416274547577, std err: 0.46%
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.04it/s]Batch 1: Zero-shot top-1 Accuracy: 64.06%, std err: 6.00%
Batch 1: Avg Cosine Similarity Score: 28.98438537120819, std err: 0.32%
2it [00:00,  2.66it/s]                                                          Batch 2: Zero-shot top-1 Accuracy: 66.67%, std err: 4.81%
Batch 2: Avg Cosine Similarity Score: 29.058409055074055, std err: 0.26%
3it [00:01,  2.98it/s]Batch 3: Zero-shot top-1 Accuracy: 67.00%, std err: 4.70%
Batch 3: Avg Cosine Similarity Score: 28.966181106567383, std err: 0.26%
4it [00:01,  3.50it/s]
 Zero-shot top-1 Accuracy: 67.00%, std err: 4.70%
 Avg Cosine Similarity Score: 28.966181106567383, std err: 0.

In [37]:
!python3 evaluate.py --model_name='clip' --data='COCO-13k' --batch_size=32 --data_subset=100 --eval_run_name='test6' --max_batch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|                                                     | 0/1 [00:00<?, ?it/s]Batch 0: Zero-shot top-1 Accuracy: 65.62%, std err: 8.40%
Batch 0: Avg Cosine Similarity Score: 29.700043380260468, std err: 0.53%
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.05it/s]Batch 1: Zero-shot top-1 Accuracy: 59.38%, std err: 6.14%
Batch 1: Avg Cosine Similarity Score: 29.69073587656021, std err: 0.40%
2it [00:00,  2.67it/s]                                                          Batch 2: Zero-shot top-1 Accuracy: 62.50%, std err: 4.94%
Batch 2: Avg Cosine Similarity Score: 29.620400329430897, std err: 0.32%
3it [00:01,  3.02it/s]Batch 3: Zero-shot top-1 Accuracy: 63.00%, std err: 4.83%
Batch 3: Avg Cosine Similarity Score: 29.574338397979737, std err: 0.32%
4it [00:01,  3.54it/s]
 Zero-shot top-1 Accuracy: 63.00%, std err: 4.83%
 Avg Cosine Similarity Score: 29.574338397979737, std err: 0.32%
